In [444]:
import skimage.io as io
from skimage.color import rgb2gray,rgb2hsv,hsv2rgb
from skimage.exposure import histogram
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
import matplotlib.pyplot as plt
from matplotlib.pyplot import bar
import numpy as np
import os
import pandas as pd

## Color Features

#### Extract color features ( HSV Color Model )

In [445]:
def extract_hsv_features(image):
    img_hsv = (rgb2hsv(image)*255).astype(np.uint8)

    h_hist,h_bins = histogram(img_hsv[:,:,0], nbins=256)
    s_hist,s_bins = histogram(img_hsv[:,:,1], nbins=256)
    v_hist,v_bins = histogram(img_hsv[:,:,2], nbins=256)

    h_bins = h_bins.astype(np.uint8)
    s_bins = s_bins.astype(np.uint8)
    v_bins = v_bins.astype(np.uint8)

    hue = np.zeros(256,dtype=int)
    saturation = np.zeros(256,dtype=int)
    value = np.zeros(256,dtype=int)

    for i in range(h_bins.size):
        hue[h_bins[i]]=h_hist[i]

    for i in range(s_bins.size):
        saturation[s_bins[i]]=s_hist[i]

    for i in range(v_bins.size):
        value[v_bins[i]]=v_hist[i]

    histograms=hue.tolist()+saturation.tolist()+value.tolist()

    return histograms

In [446]:
def create_column_hsv_feature(nbins):
    labels = np.zeros(3*nbins+1, dtype='<U20')
    for i in range(nbins):
        labels[i] = f'Hue_{i}'
        labels[i+256] = f'Saturation_{i}'
        labels[i+512] = f'Value_{i}'
        labels[256*3]='label'
    return labels

In [447]:
columns = create_column_hsv_feature(256)

data =[]

base_path = 'archive/Train/Train'
categories = ['Healthy','Powdery','Rust']
for category in categories:
    images = os.listdir(f'{base_path}/{category}')
    
    for image in images:
        image_path = f'{base_path}/{category}/{image}'
        img = io.imread(image_path)
        
        img_features = extract_hsv_features(img)

        if category == 'Healthy':
            img_features.append(0)
        elif category == 'Powdery':
            img_features.append(1)
        else:
            img_features.append(2)
    
        data.append(img_features)

pd.DataFrame(data,columns=columns).to_csv('Features/hsv_features.csv',index=False)

#### Extract color features ( RGB Color Model )

In [448]:
def extract_rgb_features(image):
    
    r_hist,r_bins = histogram(image[:,:,0], nbins=256)
    g_hist,g_bins = histogram(image[:,:,1], nbins=256)
    b_hist,b_bins = histogram(image[:,:,2], nbins=256)

    r_bins = r_bins.astype(np.uint8)
    g_bins = g_bins.astype(np.uint8)
    b_bins = b_bins.astype(np.uint8)

    red = np.zeros(256,dtype=int)
    green = np.zeros(256,dtype=int)
    blue = np.zeros(256,dtype=int)

    for i in range(r_bins.size):
        red[r_bins[i]]=r_hist[i]

    for i in range(g_bins.size):
        green[g_bins[i]]=g_hist[i]
    
    for i in range(b_bins.size):
        blue[b_bins[i]]=b_hist[i]

    histograms=red.tolist()+green.tolist()+blue.tolist()

    return histograms

In [449]:
def create_column_rgb_feature(nbins):
    labels = np.zeros(3*nbins+1, dtype='<U20')
    for i in range(nbins):
        labels[i] = f'Red_{i}'
        labels[i+256] = f'Green_{i}'
        labels[i+512] = f'Blue_{i}'
    labels[256*3]='label'
    return labels

In [450]:
columns = create_column_rgb_feature(256)

data =[]

base_path = 'archive/Train/Train'
categories = ['Healthy','Powdery','Rust']

for category in categories:
    images = os.listdir(f'{base_path}/{category}')
    for image in images:
        image_path = f'{base_path}/{category}/{image}'
        img = io.imread(image_path)
        img_features = extract_rgb_features(img)

        if category == 'Healthy':
            img_features.append(0)
        elif category == 'Powdery':
            img_features.append(1)
        else:
            img_features.append(2)
    
        data.append(img_features)

pd.DataFrame(data,columns=columns).to_csv('Features/rgb_features.csv',index=False)

## GLCM


In [451]:
def extract_glcm_feature(image):
    gray_image = (rgb2gray(image)*255).astype(np.uint8)
    glcm = graycomatrix(gray_image,distances = [1],angles=[0],levels=256,symmetric=True)

    contrast = graycoprops(glcm,prop='contrast')
    dissimilarity = graycoprops(glcm,prop='dissimilarity')
    homogeneity = graycoprops(glcm,prop='homogeneity')
    asm = graycoprops(glcm,prop='ASM')
    energy = graycoprops(glcm,prop='energy')
    correlation = graycoprops(glcm,prop='correlation')

    feature = [contrast[0][0],dissimilarity[0][0],homogeneity[0][0],asm[0][0],energy[0][0],correlation[0][0]]
    
    return feature


In [452]:
def create_column_glcm_feature():
    labels = ['contrast','dissimilarity','homogeneity','asm','energy','correlation','label']
    return labels

In [453]:
columns = create_column_glcm_feature()

data =[]

base_path = 'archive/Train/Train'
categories = ['Healthy','Powdery','Rust']

for category in categories:
    images = os.listdir(f'{base_path}/{category}')
    for image in images:

        image_path = f'{base_path}/{category}/{image}'
        img = io.imread(image_path)
        
        img_features = extract_glcm_feature(img)

        if category == 'Healthy':
            img_features.append(0)
        elif category == 'Powdery':
            img_features.append(1)
        else:
            img_features.append(2)

        data.append(img_features)

pd.DataFrame(data,columns=columns).to_csv('Features/glcm_features.csv',index=False)

## LBP

In [454]:
def custom_local_binary_pattern(image):
    R=1 # Radius
    P=8 # Number of neighbors

    # Compute the dimensions of the padded image
    pad_width = R
    padded_image = np.pad(image, pad_width, mode='constant', constant_values=0)
    
    # Initialize the LBP image
    lbp_image = np.zeros_like(image)
    
    # Iterate over each pixel in the image
    for i in range(R, padded_image.shape[0] - R):
        for j in range(R, padded_image.shape[1] - R):
            center_value = padded_image[i,j]
            # Extract the region of interest
            region = padded_image[i - R:i + R + 1, j - R:j + R + 1]
            
            # Compute the binary pattern
            binary_pattern = (region > center_value) * 1
            
            # Compute the decimal value of the binary pattern            
            binary_string=''
            for k in range(3):
                binary_string+=str(binary_pattern[0,k])
            binary_string+=str(binary_pattern[1,2])
            for k in range(2,-1,-1):
                binary_string+=str(binary_pattern[2,k])
            binary_string+=str(binary_pattern[1,0])
                    
            
            # Store the decimal value in the LBP image
            lbp_image[i - R, j - R] = int(binary_string, 2)
    
    return lbp_image

In [455]:
def extract_lbp_feature(image):
    gray_image = (rgb2gray(image)*255).astype(np.uint8)
    lbp_image = local_binary_pattern(gray_image,P=8,R=1,method='default')

    lbp_hist,lbp_bins = histogram(lbp_image, nbins=256)
    lbp_bins = lbp_bins.astype(np.uint8)

    lbp = np.zeros(256,dtype=int)

    for i in range(lbp_bins.size):
        lbp[lbp_bins[i]]=lbp_hist[i]

    return lbp.tolist()

In [456]:
columns = [f'LBP_Bin_{i}' for i in range(256)]
columns.append('label')

data =[]

base_path = 'archive/Train/Train'
categories = ['Healthy','Powdery','Rust']

for category in categories:
    images = os.listdir(f'{base_path}/{category}')
    for image in images:
        image_path = f'{base_path}/{category}/{image}'
        img = io.imread(image_path)
        
        img_features = extract_lbp_feature(img)

        if category == 'Healthy':
            img_features.append(0)
        elif category == 'Powdery':
            img_features.append(1)
        else:
            img_features.append(2)

        data.append(img_features)

pd.DataFrame(data,columns=columns).to_csv('Features/lbp_features.csv',index=False)

## Gabor filters

In [457]:
def create_column_gabor_feature(frequencies,orientations):
    columns = []
    for freq in frequencies:
        for theta in orientations:
            columns.append(f'Gabor_mean_freq{freq}_theta{theta}')
            columns.append(f'Gabor_std_freq{freq}_theta{theta}')
    return columns


def extract_gabor_features(image,frequencies,orientations):
    gray_image = (rgb2gray(image)*255).astype(np.uint8)
    features = []
    for freq in frequencies:
        for theta in orientations:
            print((freq,theta))
            real,imaginary = gabor(gray_image, frequency=freq,theta=theta)
            features.append(real.mean())
            features.append(real.std())
    return features



In [ ]:
frequencies = [0.2,0.4,0.6,0.8,1]
orientations = [0,45,90,135]

columns = create_column_gabor_feature(frequencies,orientations)
columns.append('label')

data =[]

base_path = 'archive/Train/Train'
categories = ['Healthy','Powdery','Rust']

for category in categories:
    images = os.listdir(f'{base_path}/{category}')
    for image in images:
        image_path = f'{base_path}/{category}/{image}'
        img = io.imread(image_path)
        
        img_features = extract_gabor_features(img,frequencies,orientations)

        if category == 'Healthy':
            img_features.append(0)
        elif category == 'Powdery':
            img_features.append(1)
        else:
            img_features.append(2)

        data.append(img_features)

pd.DataFrame(data,columns=columns).to_csv('Features/gabor_features.csv',index=False)

(0.2, 0)
(0.2, 45)
(0.2, 90)
(0.2, 135)
(0.4, 0)
(0.4, 45)
(0.4, 90)
(0.4, 135)
(0.6, 0)
(0.6, 45)
(0.6, 90)
(0.6, 135)
(0.8, 0)
(0.8, 45)
(0.8, 90)
(0.8, 135)
(1, 0)
(1, 45)
(1, 90)
(1, 135)
(0.2, 0)
(0.2, 45)
(0.2, 90)
(0.2, 135)
(0.4, 0)
(0.4, 45)
(0.4, 90)
(0.4, 135)
(0.6, 0)
(0.6, 45)
(0.6, 90)
(0.6, 135)
(0.8, 0)
(0.8, 45)
(0.8, 90)
(0.8, 135)
(1, 0)
(1, 45)
(1, 90)
(1, 135)
(0.2, 0)
(0.2, 45)
(0.2, 90)
(0.2, 135)
(0.4, 0)
(0.4, 45)
(0.4, 90)
(0.4, 135)
(0.6, 0)
(0.6, 45)
(0.6, 90)
(0.6, 135)
(0.8, 0)
(0.8, 45)
(0.8, 90)
(0.8, 135)
(1, 0)
(1, 45)
(1, 90)
(1, 135)
(0.2, 0)
(0.2, 45)
(0.2, 90)
(0.2, 135)
(0.4, 0)
(0.4, 45)
(0.4, 90)
(0.4, 135)
(0.6, 0)
(0.6, 45)
(0.6, 90)
(0.6, 135)
(0.8, 0)
(0.8, 45)
(0.8, 90)
(0.8, 135)
(1, 0)
(1, 45)
(1, 90)
(1, 135)
(0.2, 0)
(0.2, 45)
(0.2, 90)
(0.2, 135)
(0.4, 0)
(0.4, 45)
(0.4, 90)
(0.4, 135)
(0.6, 0)
(0.6, 45)
(0.6, 90)
(0.6, 135)
(0.8, 0)
(0.8, 45)
(0.8, 90)
(0.8, 135)
(1, 0)
(1, 45)
(1, 90)
(1, 135)
(0.2, 0)
(0.2, 45)
(0.2, 90)
(0.2, 135)
